In [ ]:
import requests
import pandas as pd
import json
import urllib

address_list = [' ']
columns = ["zoneName", "address", "latitude", "longitude","type"]
api_key = "OniyFz9aRuv6SpCF+x2ytMHyEeMTJ9DXHyWdz+JL6RnGVrrEXR+aeKYZ0+4xah26a3Kuk0TTspLZ7MACxZObgg=="
url = 'http://apis.data.go.kr/1613000/CarSharingInfoService/getCarZoneListByName'

df1_1 = pd.DataFrame()

for address in address_list:
    params ={'serviceKey' : api_key, 'pageNo' : '1', 'numOfRows' : '5000', '_type' : 'json', 'zoneName' : address }
    response = requests.get(url, params=params)
    result=json.loads(response.content.decode('utf-8'))

    if "response" in result and "body" in result["response"] and "items" in result["response"]["body"] and "item" in result["response"]["body"]["items"]:
        item = result["response"]["body"]["items"]["item"]
        if type(item) == dict :
            item = [item]
            temp_df = pd.DataFrame(item)
        elif type(item) == list :
            temp_df = pd.DataFrame(item)

        df1_1 = pd.concat([df1_1, temp_df], ignore_index=True)
    else :
        continue  
df1_1 = df1_1[columns]
df1_1 = df1_1[df1_1["type"]==2]
# 운행종료 제거
df1_1_filtered = df1_1[~df1_1["zoneName"].str.contains(r"운영종료", na=False)].drop_duplicates()

#두 번째 데이터셋
df1_2 = pd.read_csv("./전국렌터카업체정보표준데이터.csv", encoding="euc-kr")
def choose_address(row):
    # 둘 다 존재하면 도로명 주소 사용
    if pd.notna(row["차고지도로명주소"]) and pd.notna(row["차고지지번주소"]):
        return row["차고지지번주소"]
    # 하나만 존재하면 해당 주소 사용
    elif pd.notna(row["차고지도로명주소"]):
        return row["차고지도로명주소"]
    elif pd.notna(row["차고지지번주소"]):
        return row["차고지지번주소"]
    # 둘 다 없으면 NaN
    else:
        return None

# 각 행에 대해 choose_address 함수 적용
df1_2["address"] = df1_2.apply(choose_address, axis=1)

columns = ['제공기관명', "address", '위도', '경도']
df1_2_filtered = df1_2[df1_2["업체명"].str.contains(r"그린카", na=False)]
df1_2_filtered = df1_2_filtered[columns]
df1_2_filtered.rename(columns={"제공기관명":"zoneName",
                              "위도":"latitude",
                              "경도":"longitude",
                              }, inplace=True)

df1_greencar = pd.concat([df1_1_filtered, df1_2_filtered], ignore_index=True)
df1_greencar["latitude"] = df1_greencar["latitude"].round(3)
df1_greencar["longitude"] = df1_greencar["longitude"].round(3)
df1_greencar.drop(columns = ["type"], inplace=True)

# 위도, 경도 기준으로 중복 제거
df1_greencar = df1_greencar.drop_duplicates(subset=["latitude", "longitude"]).reset_index(drop=True)
df1_greencar.to_csv("carSharingPlace.csv", encoding="utf-8")